In [ ]:
!pip install ollama
!pip install pdfplumber

---
# ollama generate로 모델 반응 확인

In [2]:
import ollama

prompt = "안녕. AI를 한국어로 짧고 간결하게 설명해줘."
response = ollama.generate(model='qwen2.5', prompt=prompt)

response

GenerateResponse(model='qwen2.5', created_at='2025-05-14T01:43:41.7234015Z', done=True, done_reason='stop', total_duration=4015404800, load_duration=40968600, prompt_eval_count=48, prompt_eval_duration=125000000, eval_count=34, eval_duration=3848000000, response='안녕하세요. AI는 인공지능의 줄임말로, 컴퓨터가 인간과 같은 능력을 가진 기술입니다.🤖💡', context=[151644, 8948, 198, 2610, 525, 1207, 16948, 11, 3465, 553, 54364, 14817, 13, 1446, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 126246, 144370, 13, 15235, 18411, 130092, 31079, 17380, 47911, 100, 34395, 16778, 226, 88781, 128555, 133828, 33883, 144131, 13, 151645, 198, 151644, 77091, 198, 126246, 144370, 91145, 13, 15235, 16560, 58677, 78125, 21329, 66019, 20401, 53989, 226, 93701, 127452, 17380, 11, 143522, 19969, 133204, 53680, 129381, 143861, 98, 135818, 35509, 85251, 54116, 125880, 78952, 13, 147238, 145289])

In [27]:
def make_prompt(input_text):
    prompt = f"""
    문서 평가를 수행하여 핵심 내용을 추출하고 잘 작성된 부분과 잘못된 부분을 식별한 후 점수를 매깁니다. 문서 점수는 100점 만점이며, 모든 문서는 100점에서 시작하며 잘못된 부분이 발견될 때마다 점수를 차감합니다.

    
    # 단계

    1. **핵심 내용 추출:** 문서의 주요 메시지나 주장을 식별합니다.
    2. **잘 작성된 부분 식별:** 논리적 구조, 어휘 사용, 문법적 정확성과 같은 점에서 문서의 장점을 파악합니다.
    3. **잘못된 부분 식별:** 논리적 불일치, 부정확한 정보, 불명확한 표현 등을 확인합니다.
    4. **점수 조정:** 잘못된 부분마다 점수를 깎고 그 이유를 명시합니다.
    5. **최종 평가:** 최종 점수와 함께 문서의 강점과 개선할 점을 요약합니다.

    # 출력 형식
    - 한글로 출력합니다.
    - 문서의 핵심 내용을 1-2 문장으로 요약합니다.
    - 잘 작성된 부분에 대한 평가와 그 이유를 나열합니다.
    - 잘못된 부분에 대한 평가와 그 이유를 나열합니다.
    - 잘못된 부분에 따라 몇 점이 차감되었는지 명시합니다.
    - 최종 문서 점수를 제공합니다.
    - 전체적인 평가서를 문단 형식으로 작성합니다.

    # 예시

    **예시 1**:
    - **요약**: 이 문서는 기후 변화가 해양 생물에 미치는 영향을 논의하며, 생물 다양성의 상실과 해양 화학의 변화를 강조합니다.
    - **잘 쓰여진 부품**:
    - "해양 산성화에 관한 부분은 철저히 조사되고 명확하게 설명되어 있습니다."
    - **부실하게 작성된 부품**:
    - "서론에 명확한 논지가 부족합니다." (-5점)
    - "일부 데이터는 인용 없이 제공됩니다." (-10점)
    - **점수**: 85
    - **최종 평가**: 이 문서는 유익하고 잘 연구되었지만 신뢰성을 높이기 위해 구조와 인용을 개선해야 합니다.
    
    **입력:** {input_text}

    **출력:**

    ```
    - 핵심 내용 요약: [문서의 핵심 내용을 간단히 요약합니다.]
    - 잘 작성된 부분 평가:
    - [구체적인 부분]이 [이유]로 인해 잘 작성되었습니다.
    - ...
    - 잘못된 부분 평가:
    - [구체적인 부분]이 [문제점]으로 인해 잘못되었습니다.
    - ...
    - 차감된 점수: [잘못된 부분별로 차감된 점수 명시]
    - 최종 점수: [차감 후 최종 점수]
    - 종합 평가 요약: 문서에서 잘 된 부분과 개선할 부분에 대한 평가를 종합하여 진술합니다.
    ```

    # 메모

    - 문법적, 논리적, 내용적 오류를 감지할 때 주의하십시오.
    - 점수 차감 시 이유를 명확하고 구체적으로 설명합니다.
    - 전반적인 분석과 평가에 객관적이고 공정하게 접근합니다.
    """
    return prompt

In [3]:
import pdfplumber

def table_to_markdown(table):
    """
    2차원 리스트(table)를 마크다운 표 문자열로 변환
    """
    if not table or not table[0]:
        return ""
    # 헤더와 구분선
    header = "| " + " | ".join(table[0]) + " |"
    separator = "| " + " | ".join(['---'] * len(table[0])) + " |"
    # 데이터 행
    rows = ["| " + " | ".join(row) + " |" for row in table[1:]]
    return "\n".join([header, separator] + rows)

def extract_text_from_pdf(pdf_path):
    result = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages, start=1):
            text = ""
            text += page.extract_text() + "\n"
            tables = page.extract_tables()
            if tables:
                for table in tables:
                    md_table = table_to_markdown(table)
                    text += f"\n```markdown\n{md_table}\n```"
                    
            result.append({
                'page':page_num,
                'text': text
            })
                
    return result


In [4]:
result = extract_text_from_pdf('./data/1.pdf')
result


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


[{'page': 1,
  'text': '과학기술정보통신부 공고 제2024-0531호\n2024년 초거대 AI 기반 플랫폼 이용지원 사업공고\n과학기술정보통신부와 한국지능정보사회진흥원(전담기관)은\n『2024년 초거대 AI 기반 플랫폼 이용지원』사업 공모를 다음과\n같이 추진하오니 많은 참여바랍니다.\n과학기술정보통신부\n한국지능정보사회진흥원\n사업내용 및 제안서 서식, 협약 등 문의 : AI서비스혁신팀 (☏ 053-230-1407, 1492, 1468)\n제안서 접수 후 평가 관련 문의 : 재무관리팀 (☏ 053-230-1180, 1176)\n★ 이 공모와 관련하여 공모공고서와 공모안내서의 내용이 서로 다를 경우에는 공모공고서의\n내용을 우선하여 적용하며 제안서 접수 및 평가 이후에도 제안사의 참여 적격 여부를 사후\n판정할 수 있습니다.\n★ 공동수급비율 : 전체금액 중 해당기관 사업비(정부출연금+매칭금액), 공동수급비율은 합계\n가 반드시 100%로 구성되어야 제출 가능\n★ PC 및 네트워크 등 사용자 환경에 따라 수행계획서 제출의 어려움이 발생할 수 있으므로\n수행계획서는 가급적 마감 전일까지 제출하여 주시기 바랍니다.\n1. 일반사항\n가. 공 모 명 : 2024년 초거대 AI 기반 플랫폼 이용지원\n나. 선정방법 : 공모\n다. 사업예산 : 2,700백만원(정부출연금 기준, 정부출연금 및 민간부담금(매칭펀드방식))\n- 총 사업비는 정부출연금 및 참여 컨소시엄 부담금(현금+현물)으로 구성\n라. 협약기간 : 협약체결일 ∼ 2024. 12. 15.\n마. 공고일정(방문접수는 허용하지 않습니다.)\n- 1 -\n'},
 {'page': 2,
  'text': '구 분 공고기간 사전 등록(필수) 참가신청(온라인) 제안서 평가\n2024.5.14(화) 2024.5.14(화)\n날 짜 ~ 6.4(화) 10:00 6.10(월) 예정\n~ 6.4(화) ~ 5.24(금)\nNIA\nnia.or.kr 공지(nia.or.kr), 메일접수\n장 소 대구본원\nmsi

### qwen2.5:7b

In [26]:
import ollama
response = ollama.generate(model='qwen2.5', prompt=make_prompt(result))

In [9]:
print(response.response)

- 핵심 내용 요약: 이 문서는 제안서와 수행계획서의 평가 절차, 사업자 선정, 협약 체결, 비밀유지 의무, 정부권장정책 법적 의무구매 준수 등을 설명하고 있습니다.
- 잘 작성된 부분 평가:
    - **구체적인 부분**: 문서 구조와 명확한 분류 (예: 1. 사업자 선정 절차 2. 협약 체결 등)  
      **이유**: 이러한 구조적 접근은 내용을 이해하기 쉽고, 읽는 이에게 필요한 정보를 효과적으로 전달합니다.
- 잘못된 부분 평가:
    - **구체적인 부분**: 일부 문장에서 부정확한 표현이나 번역 오류 (예: "과제조정대상자는 최종 선정 사업자가 아니며 NIA의 "ICT 기금사업 및 연구개발사업 관리 지침“에 의거 최종 사업자 선정에서 제외될 수 있음")  
      **문제점**: "관리지침" 대신 "지침"으로 수정하면 문장이 더 명확해집니다. 그리고 "“"는 따옴표로 바꿔야 합니다.
    - **구체적인 부분**: 몇몇 문장의 구조가 불완전하거나 부자연스럽습니다 (예: "※ 과제조정대상자는 최종 선정 사업자가 아니며 NIA의 "ICT 기금사업 및 연구개발사업 관리 지침“에 의거 최종 사업자 선정에서 제외될 수 있음")  
      **문제점**: "최종 선정 사업자가 아니며"와 "NIA의 'ICT 기금사업 및 연구개발사업 관리 지침'에 의거" 사이에 쉼표나 문장 부호가 필요합니다.
- 차감된 점수: 
    - "과제조정대상자는 최종 선정 사업자가 아니며 NIA의 'ICT 기금사업 및 연구개발사업 관리 지침“에 의거 최종 사업자 선정에서 제외될 수 있음" 부분에서 1점 차감
- 최종 점수: 95 (100 - 1)
- 종합 평가 요약: 문서는 구조적이고 명확하게 작성되어 있으며, 정보 전달이 효율적이지만 몇 가지 문장과 표현에서 부정확한 부분들이 발견되었습니다. 특히 문장 구조와 문법적인 정정이 필요합니다.


### qwen3:30b-a3b

In [28]:
import ollama
response = ollama.generate(model='qwen3:30b-a3b', prompt=make_prompt(result))

In [29]:
print(response.response)

<think>
Okay, let's start by understanding what the user is asking for. They want me to evaluate a document based on specific steps: extracting the main content, identifying well-written parts, pointing out flaws, adjusting the score, and providing a final evaluation. The document here is a public notice about a government project called "Super AI Platform Utilization Support Project" by the Korea Institute of Information Society (NIA). 

First, I need to summarize the core content. The document outlines the application process, required documents, evaluation criteria, and other important details for participating in the project. It's a formal notice, so the structure is likely to be quite detailed.

Next, the well-written parts. I should look for clarity, organization, and completeness. For example, the section on submission requirements is detailed, specifying that all files must be in PDF and uploaded as a single file. That's good because it reduces confusion. Also, the list of requ